In [1]:
import seaborn as sns
import pandas as pd
import numpy as np

df=sns.load_dataset("diamonds")
df

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


In [2]:
q1=df.price.quantile(0.25)
q3=df.price.quantile(0.75)
iqr=q3-q1
down_limit=(q1-iqr*1.5)
up_limit=(q3+iqr*1.5)
down_limit,up_limit

(-5611.375, 11885.625)

In [3]:
df=df[df.price < 11886]

In [4]:
df_one_hot=pd.get_dummies(df,columns=["cut","color","clarity"])

In [5]:
df=df_one_hot

In [6]:
from sklearn.neighbors import LocalOutlierFactor
lof=LocalOutlierFactor()
lof.fit_predict(df)
scores=lof.negative_outlier_factor_

In [7]:
vector= scores >= -2

In [8]:
df=df[vector]

In [9]:
x=df.drop(["price"],axis=1)
y=df.price

In [10]:
from sklearn.model_selection import train_test_split,GridSearchCV

In [11]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20)
y_test

28653      675
41335     1215
18553     7564
46008     1726
30576      734
         ...  
32827      806
29268      698
23647    11681
12907     5375
10371     4775
Name: price, Length: 10060, dtype: int64

In [12]:
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale 
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import neighbors
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

In [15]:
lgbm_params={
    "learning_rate":[0.01,0.1,0.2],
    "num_leaves":[25,31,40],
    "n_estimators":[50,100,500,2000],
    "min_child_samples":[10,20,30]
}

lgbm_cv_model=GridSearchCV(LGBMRegressor(),lgbm_params,cv=5,verbose=2,n_jobs=-1).fit(x_train,y_train)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:  5.5min finished


In [16]:
lgbm_cv_model.best_params_

{'learning_rate': 0.01,
 'min_child_samples': 20,
 'n_estimators': 2000,
 'num_leaves': 40}

In [17]:
lgbm_tuned=LGBMRegressor(learning_rate=0.01,
                        min_child_samples=20,
                        n_estimators=2000,
                        num_leaves=40).fit(x_train,y_train,eval_metric="rmse",
                                          eval_set=[(x_test,y_test)],early_stopping_rounds=2000)


[1]	valid_0's rmse: 2732.29	valid_0's l2: 7.46541e+06
Training until validation scores don't improve for 2000 rounds
[2]	valid_0's rmse: 2706.45	valid_0's l2: 7.32487e+06
[3]	valid_0's rmse: 2680.91	valid_0's l2: 7.18727e+06
[4]	valid_0's rmse: 2655.61	valid_0's l2: 7.05229e+06
[5]	valid_0's rmse: 2630.58	valid_0's l2: 6.91995e+06
[6]	valid_0's rmse: 2605.83	valid_0's l2: 6.79036e+06
[7]	valid_0's rmse: 2581.25	valid_0's l2: 6.66284e+06
[8]	valid_0's rmse: 2557.07	valid_0's l2: 6.53861e+06
[9]	valid_0's rmse: 2533.07	valid_0's l2: 6.41642e+06
[10]	valid_0's rmse: 2509.32	valid_0's l2: 6.29669e+06
[11]	valid_0's rmse: 2485.81	valid_0's l2: 6.17926e+06
[12]	valid_0's rmse: 2462.47	valid_0's l2: 6.06375e+06
[13]	valid_0's rmse: 2439.46	valid_0's l2: 5.95095e+06
[14]	valid_0's rmse: 2416.59	valid_0's l2: 5.8399e+06
[15]	valid_0's rmse: 2394.07	valid_0's l2: 5.73156e+06
[16]	valid_0's rmse: 2371.7	valid_0's l2: 5.62496e+06
[17]	valid_0's rmse: 2349.58	valid_0's l2: 5.52052e+06
[18]	valid_0'

[160]	valid_0's rmse: 748.213	valid_0's l2: 559823
[161]	valid_0's rmse: 743.714	valid_0's l2: 553111
[162]	valid_0's rmse: 739.396	valid_0's l2: 546706
[163]	valid_0's rmse: 735.034	valid_0's l2: 540275
[164]	valid_0's rmse: 730.74	valid_0's l2: 533980
[165]	valid_0's rmse: 726.591	valid_0's l2: 527934
[166]	valid_0's rmse: 722.351	valid_0's l2: 521791
[167]	valid_0's rmse: 718.247	valid_0's l2: 515879
[168]	valid_0's rmse: 714.165	valid_0's l2: 510031
[169]	valid_0's rmse: 710.08	valid_0's l2: 504213
[170]	valid_0's rmse: 706.133	valid_0's l2: 498624
[171]	valid_0's rmse: 702.133	valid_0's l2: 492990
[172]	valid_0's rmse: 698.292	valid_0's l2: 487611
[173]	valid_0's rmse: 694.437	valid_0's l2: 482242
[174]	valid_0's rmse: 690.595	valid_0's l2: 476921
[175]	valid_0's rmse: 686.897	valid_0's l2: 471827
[176]	valid_0's rmse: 683.241	valid_0's l2: 466819
[177]	valid_0's rmse: 679.6	valid_0's l2: 461856
[178]	valid_0's rmse: 675.972	valid_0's l2: 456938
[179]	valid_0's rmse: 672.42	valid_

[373]	valid_0's rmse: 405.331	valid_0's l2: 164293
[374]	valid_0's rmse: 405.002	valid_0's l2: 164027
[375]	valid_0's rmse: 404.67	valid_0's l2: 163758
[376]	valid_0's rmse: 404.303	valid_0's l2: 163461
[377]	valid_0's rmse: 403.92	valid_0's l2: 163151
[378]	valid_0's rmse: 403.592	valid_0's l2: 162886
[379]	valid_0's rmse: 403.247	valid_0's l2: 162608
[380]	valid_0's rmse: 402.897	valid_0's l2: 162326
[381]	valid_0's rmse: 402.579	valid_0's l2: 162070
[382]	valid_0's rmse: 402.242	valid_0's l2: 161799
[383]	valid_0's rmse: 401.919	valid_0's l2: 161538
[384]	valid_0's rmse: 401.561	valid_0's l2: 161251
[385]	valid_0's rmse: 401.221	valid_0's l2: 160978
[386]	valid_0's rmse: 400.909	valid_0's l2: 160728
[387]	valid_0's rmse: 400.642	valid_0's l2: 160514
[388]	valid_0's rmse: 400.309	valid_0's l2: 160247
[389]	valid_0's rmse: 399.979	valid_0's l2: 159983
[390]	valid_0's rmse: 399.675	valid_0's l2: 159740
[391]	valid_0's rmse: 399.343	valid_0's l2: 159475
[392]	valid_0's rmse: 399.044	val

[581]	valid_0's rmse: 371.042	valid_0's l2: 137672
[582]	valid_0's rmse: 370.966	valid_0's l2: 137616
[583]	valid_0's rmse: 370.925	valid_0's l2: 137586
[584]	valid_0's rmse: 370.887	valid_0's l2: 137557
[585]	valid_0's rmse: 370.832	valid_0's l2: 137516
[586]	valid_0's rmse: 370.731	valid_0's l2: 137442
[587]	valid_0's rmse: 370.64	valid_0's l2: 137374
[588]	valid_0's rmse: 370.605	valid_0's l2: 137348
[589]	valid_0's rmse: 370.522	valid_0's l2: 137286
[590]	valid_0's rmse: 370.453	valid_0's l2: 137236
[591]	valid_0's rmse: 370.393	valid_0's l2: 137191
[592]	valid_0's rmse: 370.313	valid_0's l2: 137132
[593]	valid_0's rmse: 370.244	valid_0's l2: 137081
[594]	valid_0's rmse: 370.196	valid_0's l2: 137045
[595]	valid_0's rmse: 370.116	valid_0's l2: 136986
[596]	valid_0's rmse: 370.067	valid_0's l2: 136950
[597]	valid_0's rmse: 370.005	valid_0's l2: 136903
[598]	valid_0's rmse: 369.934	valid_0's l2: 136851
[599]	valid_0's rmse: 369.872	valid_0's l2: 136806
[600]	valid_0's rmse: 369.806	va

[813]	valid_0's rmse: 362.717	valid_0's l2: 131564
[814]	valid_0's rmse: 362.689	valid_0's l2: 131543
[815]	valid_0's rmse: 362.663	valid_0's l2: 131524
[816]	valid_0's rmse: 362.631	valid_0's l2: 131502
[817]	valid_0's rmse: 362.596	valid_0's l2: 131476
[818]	valid_0's rmse: 362.564	valid_0's l2: 131453
[819]	valid_0's rmse: 362.528	valid_0's l2: 131427
[820]	valid_0's rmse: 362.489	valid_0's l2: 131399
[821]	valid_0's rmse: 362.467	valid_0's l2: 131382
[822]	valid_0's rmse: 362.436	valid_0's l2: 131360
[823]	valid_0's rmse: 362.418	valid_0's l2: 131347
[824]	valid_0's rmse: 362.402	valid_0's l2: 131336
[825]	valid_0's rmse: 362.405	valid_0's l2: 131338
[826]	valid_0's rmse: 362.388	valid_0's l2: 131325
[827]	valid_0's rmse: 362.366	valid_0's l2: 131309
[828]	valid_0's rmse: 362.353	valid_0's l2: 131299
[829]	valid_0's rmse: 362.327	valid_0's l2: 131281
[830]	valid_0's rmse: 362.299	valid_0's l2: 131261
[831]	valid_0's rmse: 362.266	valid_0's l2: 131237
[832]	valid_0's rmse: 362.262	v

...

[1188]	valid_0's rmse: 357.296	valid_0's l2: 127661
[1189]	valid_0's rmse: 357.292	valid_0's l2: 127657
[1190]	valid_0's rmse: 357.306	valid_0's l2: 127668
[1191]	valid_0's rmse: 357.302	valid_0's l2: 127664
[1192]	valid_0's rmse: 357.292	valid_0's l2: 127657
[1193]	valid_0's rmse: 357.298	valid_0's l2: 127662
[1194]	valid_0's rmse: 357.284	valid_0's l2: 127652
[1195]	valid_0's rmse: 357.271	valid_0's l2: 127643
[1196]	valid_0's rmse: 357.266	valid_0's l2: 127639
[1197]	valid_0's rmse: 357.261	valid_0's l2: 127635
[1198]	valid_0's rmse: 357.261	valid_0's l2: 127635
[1199]	valid_0's rmse: 357.276	valid_0's l2: 127646
[1200]	valid_0's rmse: 357.28	valid_0's l2: 127649
[1201]	valid_0's rmse: 357.28	valid_0's l2: 127649
[1202]	valid_0's rmse: 357.271	valid_0's l2: 127643
[1203]	valid_0's rmse: 357.258	valid_0's l2: 127633
[1204]	valid_0's rmse: 357.244	valid_0's l2: 127623
[1205]	valid_0's rmse: 357.231	valid_0's l2: 127614
[1206]	valid_0's rmse: 357.229	valid_0's l2: 127613
[1207]	valid_0

[1352]	valid_0's rmse: 356.601	valid_0's l2: 127165
[1353]	valid_0's rmse: 356.595	valid_0's l2: 127160
[1354]	valid_0's rmse: 356.585	valid_0's l2: 127153
[1355]	valid_0's rmse: 356.571	valid_0's l2: 127143
[1356]	valid_0's rmse: 356.578	valid_0's l2: 127148
[1357]	valid_0's rmse: 356.581	valid_0's l2: 127150
[1358]	valid_0's rmse: 356.568	valid_0's l2: 127140
[1359]	valid_0's rmse: 356.565	valid_0's l2: 127139
[1360]	valid_0's rmse: 356.584	valid_0's l2: 127152
[1361]	valid_0's rmse: 356.584	valid_0's l2: 127152
[1362]	valid_0's rmse: 356.593	valid_0's l2: 127158
[1363]	valid_0's rmse: 356.585	valid_0's l2: 127153
[1364]	valid_0's rmse: 356.574	valid_0's l2: 127145
[1365]	valid_0's rmse: 356.575	valid_0's l2: 127146
[1366]	valid_0's rmse: 356.587	valid_0's l2: 127155
[1367]	valid_0's rmse: 356.588	valid_0's l2: 127155
[1368]	valid_0's rmse: 356.594	valid_0's l2: 127159
[1369]	valid_0's rmse: 356.586	valid_0's l2: 127154
[1370]	valid_0's rmse: 356.572	valid_0's l2: 127143
[1371]	valid

[1562]	valid_0's rmse: 355.946	valid_0's l2: 126697
[1563]	valid_0's rmse: 355.953	valid_0's l2: 126702
[1564]	valid_0's rmse: 355.947	valid_0's l2: 126698
[1565]	valid_0's rmse: 355.947	valid_0's l2: 126698
[1566]	valid_0's rmse: 355.945	valid_0's l2: 126697
[1567]	valid_0's rmse: 355.945	valid_0's l2: 126697
[1568]	valid_0's rmse: 355.936	valid_0's l2: 126690
[1569]	valid_0's rmse: 355.93	valid_0's l2: 126686
[1570]	valid_0's rmse: 355.915	valid_0's l2: 126675
[1571]	valid_0's rmse: 355.918	valid_0's l2: 126678
[1572]	valid_0's rmse: 355.921	valid_0's l2: 126680
[1573]	valid_0's rmse: 355.918	valid_0's l2: 126677
[1574]	valid_0's rmse: 355.91	valid_0's l2: 126672
[1575]	valid_0's rmse: 355.906	valid_0's l2: 126669
[1576]	valid_0's rmse: 355.899	valid_0's l2: 126664
[1577]	valid_0's rmse: 355.901	valid_0's l2: 126666
[1578]	valid_0's rmse: 355.896	valid_0's l2: 126662
[1579]	valid_0's rmse: 355.89	valid_0's l2: 126658
[1580]	valid_0's rmse: 355.883	valid_0's l2: 126653
[1581]	valid_0'

[1780]	valid_0's rmse: 355.516	valid_0's l2: 126391
[1781]	valid_0's rmse: 355.499	valid_0's l2: 126379
[1782]	valid_0's rmse: 355.502	valid_0's l2: 126381
[1783]	valid_0's rmse: 355.499	valid_0's l2: 126380
[1784]	valid_0's rmse: 355.489	valid_0's l2: 126372
[1785]	valid_0's rmse: 355.485	valid_0's l2: 126370
[1786]	valid_0's rmse: 355.476	valid_0's l2: 126363
[1787]	valid_0's rmse: 355.473	valid_0's l2: 126361
[1788]	valid_0's rmse: 355.463	valid_0's l2: 126354
[1789]	valid_0's rmse: 355.453	valid_0's l2: 126347
[1790]	valid_0's rmse: 355.447	valid_0's l2: 126342
[1791]	valid_0's rmse: 355.449	valid_0's l2: 126344
[1792]	valid_0's rmse: 355.452	valid_0's l2: 126346
[1793]	valid_0's rmse: 355.446	valid_0's l2: 126342
[1794]	valid_0's rmse: 355.433	valid_0's l2: 126333
[1795]	valid_0's rmse: 355.424	valid_0's l2: 126326
[1796]	valid_0's rmse: 355.43	valid_0's l2: 126330
[1797]	valid_0's rmse: 355.432	valid_0's l2: 126332
[1798]	valid_0's rmse: 355.429	valid_0's l2: 126329
[1799]	valid_

[1960]	valid_0's rmse: 355.182	valid_0's l2: 126154
[1961]	valid_0's rmse: 355.182	valid_0's l2: 126154
[1962]	valid_0's rmse: 355.178	valid_0's l2: 126152
[1963]	valid_0's rmse: 355.167	valid_0's l2: 126143
[1964]	valid_0's rmse: 355.158	valid_0's l2: 126137
[1965]	valid_0's rmse: 355.154	valid_0's l2: 126135
[1966]	valid_0's rmse: 355.156	valid_0's l2: 126136
[1967]	valid_0's rmse: 355.156	valid_0's l2: 126136
[1968]	valid_0's rmse: 355.152	valid_0's l2: 126133
[1969]	valid_0's rmse: 355.152	valid_0's l2: 126133
[1970]	valid_0's rmse: 355.153	valid_0's l2: 126134
[1971]	valid_0's rmse: 355.155	valid_0's l2: 126135
[1972]	valid_0's rmse: 355.153	valid_0's l2: 126134
[1973]	valid_0's rmse: 355.16	valid_0's l2: 126138
[1974]	valid_0's rmse: 355.17	valid_0's l2: 126146
[1975]	valid_0's rmse: 355.152	valid_0's l2: 126133
[1976]	valid_0's rmse: 355.149	valid_0's l2: 126131
[1977]	valid_0's rmse: 355.149	valid_0's l2: 126131
[1978]	valid_0's rmse: 355.146	valid_0's l2: 126129
[1979]	valid_0

In [19]:
lgbm_tuned=LGBMRegressor(learning_rate=0.01,
                        min_child_samples=20,
                        n_estimators=1998,
                        num_leaves=40).fit(x_train,y_train)

y_pred=lgbm_tuned.predict(x_test)

print("Train setinde rmse hatası:",np.sqrt(mean_squared_error(y_train,lgbm_tuned.predict(x_train))))
print("Test setinde rmse hatası:",np.sqrt(mean_squared_error(y_test,y_pred)))
print("-"*30)
print("Train setinde r2_score:",r2_score(y_train,lgbm_tuned.predict(x_train)))
print("Test setinde r2_score",r2_score(y_test,y_pred))

Train setinde rmse hatası: 303.36419676176416
Test setinde rmse hatası: 355.080130332462
------------------------------
Train setinde r2_score: 0.9879923334502984
Test setinde r2_score 0.9834300098052274


In [20]:
### NAILED IT!!

In [33]:
test_predict=y_pred
train_predict=lgbm_tuned.predict(x_train)

train_report=pd.concat([pd.DataFrame(data=[y_train]).T,pd.DataFrame(data=[pd.Series(train_predict,index=y_train.index,name="price")]).T],axis=1)
test_report=pd.concat([pd.DataFrame(data=[y_test]).T,pd.DataFrame(data=[pd.Series(test_predict,index=y_test.index,name="price")]).T],axis=1)
train_report.columns=["true","pred"]
test_report.columns=["true","pred"]
train_report["error"]=abs(train_report.true-train_report.pred)
test_report["error"]=abs(test_report.true-test_report.pred)


In [36]:
train_report

,true,pred,error
17136,6838,6880.036375,42.036375
43452,1415,1229.234693,185.765307
61,552,626.393967,74.393967
7382,579,661.580028,82.580028
13876,5650,5160.179709,489.820291
...,...,...,...
50927,2319,2350.925826,31.925826
31086,752,752.501655,0.501655
21845,9905,8589.765889,1315.234111
21107,9278,9527.306350,249.306350


In [37]:
test_report

,true,pred,error
1998,3099,3355.651819,256.651819
51358,2364,2863.996308,499.996308
53252,2646,2692.332910,46.332910
53732,2723,2867.062601,144.062601
45257,1661,1636.674522,24.325478
...,...,...,...
29877,710,639.670364,70.329636
4338,3599,4441.469485,842.469485
32354,789,865.423146,76.423146
37499,984,877.417899,106.582101


### experiment for model improvement with pp.normalize and pp.scale

In [22]:
from sklearn import preprocessing as pp
normal=pp.normalize(x_train)
norm_xtr=pd.DataFrame(normal,columns=x_train.columns,index=x_train.index)
normal=pp.normalize(x_test)
norm_xte=pd.DataFrame(normal,columns=x_test.columns,index=x_test.index)

In [23]:
norm_xtr

,carat,depth,table,x,y,z,cut_Ideal,cut_Premium,cut_Very Good,cut_Good,...,color_I,color_J,clarity_IF,clarity_VVS1,clarity_VVS2,clarity_VS1,clarity_VS2,clarity_SI1,clarity_SI2,clarity_I1
23316,0.019804,0.712934,0.687306,0.088767,0.087952,0.054053,0.000000,0.011649,0.000000,0.000000,...,0.011649,0.0,0.000000,0.000000,0.0,0.000000,0.011649,0.000000,0.000000,0.0
33754,0.003780,0.758407,0.646231,0.053040,0.053284,0.033043,0.012193,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.012193,0.0,0.000000,0.000000,0.000000,0.000000,0.0
4644,0.010223,0.723549,0.681522,0.069402,0.068947,0.044072,0.000000,0.000000,0.000000,0.011359,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.011359,0.000000,0.0
32391,0.004653,0.732883,0.674718,0.054675,0.055024,0.034550,0.000000,0.000000,0.011633,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.011633,0.000000,0.000000,0.0
8562,0.012165,0.735773,0.666758,0.074747,0.075449,0.047258,0.000000,0.000000,0.011698,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.011698,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,0.007169,0.736027,0.669116,0.064880,0.065239,0.040027,0.000000,0.000000,0.011948,0.000000,...,0.000000,0.0,0.011948,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
40787,0.004811,0.744440,0.661457,0.057126,0.057487,0.035478,0.012026,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.012026,0.0,0.000000,0.000000,0.000000,0.000000,0.0
35562,0.004136,0.723201,0.685386,0.053649,0.053295,0.032733,0.000000,0.011817,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.011817,0.000000,0.000000,0.000000,0.0
36847,0.004301,0.711482,0.697531,0.053826,0.053361,0.032784,0.000000,0.011626,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.011626,0.000000,0.000000,0.0


In [24]:
norm_xte

,carat,depth,table,x,y,z,cut_Ideal,cut_Premium,cut_Very Good,cut_Good,...,color_I,color_J,clarity_IF,clarity_VVS1,clarity_VVS2,clarity_VS1,clarity_VS2,clarity_SI1,clarity_SI2,clarity_I1
28653,0.003920,0.719860,0.688975,0.052861,0.053336,0.032192,0.000000,0.000000,0.011879,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.011879,0.000000,0.0,0.0
41335,0.005661,0.765381,0.636835,0.058259,0.058023,0.037738,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.011793,0.000000,0.0,0.0
18553,0.013029,0.742763,0.657528,0.080243,0.080608,0.049071,0.012176,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.012176,0.000000,0.0,0.0
46008,0.006307,0.718263,0.689065,0.060848,0.061082,0.037490,0.000000,0.000000,0.011679,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.011679,0.000000,0.0,0.0
30576,0.003701,0.739035,0.668594,0.051577,0.051816,0.031997,0.011939,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.011939,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32827,0.003917,0.708707,0.700397,0.053183,0.053776,0.031933,0.000000,0.000000,0.011871,0.0,...,0.0,0.0,0.0,0.0,0.011871,0.000000,0.000000,0.000000,0.0,0.0
29268,0.003681,0.752835,0.653090,0.051297,0.050941,0.032417,0.000000,0.000000,0.011874,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.011874,0.0,0.0
23647,0.018536,0.728012,0.670698,0.090849,0.090483,0.054144,0.000000,0.000000,0.012195,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.012195,0.000000,0.0,0.0
12907,0.011908,0.741573,0.660224,0.075572,0.075218,0.047395,0.011790,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.011790,0.0,0.0


In [32]:
xtr_backup=x_train
xte_backuo=x_test

x_train=norm_xtr
x_test=norm_xte

lgbm_tuned=LGBMRegressor(learning_rate=0.01,
                        min_child_samples=20,
                        n_estimators=5000,
                        num_leaves=40).fit(x_train,y_train,eval_set=[(x_test,y_test)],eval_metric= "rmse",early_stopping_rounds=100)

#Öncesinde 5000'e kadar denedim orada da küçük bir gelişme vardı, o yüzden es ile retune yapmadım.

[1]	valid_0's rmse: 2750.84	valid_0's l2: 7.56714e+06
Training until validation scores don't improve for 100 rounds
[2]	valid_0's rmse: 2725.06	valid_0's l2: 7.42598e+06
[3]	valid_0's rmse: 2699.55	valid_0's l2: 7.2876e+06
[4]	valid_0's rmse: 2674.33	valid_0's l2: 7.15202e+06
[5]	valid_0's rmse: 2649.37	valid_0's l2: 7.01915e+06
[6]	valid_0's rmse: 2624.65	valid_0's l2: 6.8888e+06
[7]	valid_0's rmse: 2600.25	valid_0's l2: 6.76132e+06
[8]	valid_0's rmse: 2576.05	valid_0's l2: 6.63606e+06
[9]	valid_0's rmse: 2552.19	valid_0's l2: 6.51365e+06
[10]	valid_0's rmse: 2528.52	valid_0's l2: 6.39343e+06
[11]	valid_0's rmse: 2505.14	valid_0's l2: 6.27572e+06
[12]	valid_0's rmse: 2481.87	valid_0's l2: 6.15969e+06
[13]	valid_0's rmse: 2458.94	valid_0's l2: 6.04639e+06
[14]	valid_0's rmse: 2436.25	valid_0's l2: 5.93533e+06
[15]	valid_0's rmse: 2413.81	valid_0's l2: 5.82647e+06
[16]	valid_0's rmse: 2391.64	valid_0's l2: 5.71996e+06
[17]	valid_0's rmse: 2369.67	valid_0's l2: 5.61535e+06
[18]	valid_0's

[164]	valid_0's rmse: 752.361	valid_0's l2: 566046
[165]	valid_0's rmse: 748.062	valid_0's l2: 559596
[166]	valid_0's rmse: 743.925	valid_0's l2: 553425
[167]	valid_0's rmse: 739.669	valid_0's l2: 547110
[168]	valid_0's rmse: 735.666	valid_0's l2: 541204
[169]	valid_0's rmse: 731.614	valid_0's l2: 535260
[170]	valid_0's rmse: 727.488	valid_0's l2: 529239
[171]	valid_0's rmse: 723.503	valid_0's l2: 523456
[172]	valid_0's rmse: 719.608	valid_0's l2: 517835
[173]	valid_0's rmse: 715.73	valid_0's l2: 512269
[174]	valid_0's rmse: 711.877	valid_0's l2: 506769
[175]	valid_0's rmse: 708.243	valid_0's l2: 501608
[176]	valid_0's rmse: 704.575	valid_0's l2: 496426
[177]	valid_0's rmse: 700.856	valid_0's l2: 491200
[178]	valid_0's rmse: 697.153	valid_0's l2: 486023
[179]	valid_0's rmse: 693.618	valid_0's l2: 481106
[180]	valid_0's rmse: 690.132	valid_0's l2: 476282
[181]	valid_0's rmse: 686.568	valid_0's l2: 471375
[182]	valid_0's rmse: 683.094	valid_0's l2: 466618
[183]	valid_0's rmse: 679.748	va

[367]	valid_0's rmse: 423.713	valid_0's l2: 179532
[368]	valid_0's rmse: 423.347	valid_0's l2: 179223
[369]	valid_0's rmse: 422.949	valid_0's l2: 178886
[370]	valid_0's rmse: 422.539	valid_0's l2: 178539
[371]	valid_0's rmse: 422.178	valid_0's l2: 178234
[372]	valid_0's rmse: 421.772	valid_0's l2: 177892
[373]	valid_0's rmse: 421.421	valid_0's l2: 177596
[374]	valid_0's rmse: 421.066	valid_0's l2: 177296
[375]	valid_0's rmse: 420.742	valid_0's l2: 177024
[376]	valid_0's rmse: 420.409	valid_0's l2: 176744
[377]	valid_0's rmse: 420.013	valid_0's l2: 176411
[378]	valid_0's rmse: 419.681	valid_0's l2: 176132
[379]	valid_0's rmse: 419.346	valid_0's l2: 175851
[380]	valid_0's rmse: 418.952	valid_0's l2: 175520
[381]	valid_0's rmse: 418.631	valid_0's l2: 175252
[382]	valid_0's rmse: 418.263	valid_0's l2: 174944
[383]	valid_0's rmse: 417.965	valid_0's l2: 174694
[384]	valid_0's rmse: 417.646	valid_0's l2: 174428
[385]	valid_0's rmse: 417.27	valid_0's l2: 174115
[386]	valid_0's rmse: 416.932	va

[535]	valid_0's rmse: 390.226	valid_0's l2: 152276
[536]	valid_0's rmse: 390.139	valid_0's l2: 152209
[537]	valid_0's rmse: 390.036	valid_0's l2: 152128
[538]	valid_0's rmse: 389.967	valid_0's l2: 152074
[539]	valid_0's rmse: 389.903	valid_0's l2: 152024
[540]	valid_0's rmse: 389.815	valid_0's l2: 151956
[541]	valid_0's rmse: 389.732	valid_0's l2: 151891
[542]	valid_0's rmse: 389.673	valid_0's l2: 151845
[543]	valid_0's rmse: 389.592	valid_0's l2: 151782
[544]	valid_0's rmse: 389.532	valid_0's l2: 151735
[545]	valid_0's rmse: 389.425	valid_0's l2: 151652
[546]	valid_0's rmse: 389.342	valid_0's l2: 151587
[547]	valid_0's rmse: 389.269	valid_0's l2: 151531
[548]	valid_0's rmse: 389.194	valid_0's l2: 151472
[549]	valid_0's rmse: 389.104	valid_0's l2: 151402
[550]	valid_0's rmse: 389.041	valid_0's l2: 151353
[551]	valid_0's rmse: 388.929	valid_0's l2: 151266
[552]	valid_0's rmse: 388.856	valid_0's l2: 151209
[553]	valid_0's rmse: 388.782	valid_0's l2: 151152
[554]	valid_0's rmse: 388.702	v

[725]	valid_0's rmse: 379.951	valid_0's l2: 144363
[726]	valid_0's rmse: 379.935	valid_0's l2: 144351
[727]	valid_0's rmse: 379.901	valid_0's l2: 144325
[728]	valid_0's rmse: 379.866	valid_0's l2: 144298
[729]	valid_0's rmse: 379.857	valid_0's l2: 144291
[730]	valid_0's rmse: 379.832	valid_0's l2: 144272
[731]	valid_0's rmse: 379.785	valid_0's l2: 144236
[732]	valid_0's rmse: 379.743	valid_0's l2: 144205
[733]	valid_0's rmse: 379.711	valid_0's l2: 144180
[734]	valid_0's rmse: 379.673	valid_0's l2: 144152
[735]	valid_0's rmse: 379.634	valid_0's l2: 144122
[736]	valid_0's rmse: 379.576	valid_0's l2: 144078
[737]	valid_0's rmse: 379.565	valid_0's l2: 144070
[738]	valid_0's rmse: 379.54	valid_0's l2: 144050
[739]	valid_0's rmse: 379.501	valid_0's l2: 144021
[740]	valid_0's rmse: 379.49	valid_0's l2: 144013
[741]	valid_0's rmse: 379.455	valid_0's l2: 143986
[742]	valid_0's rmse: 379.409	valid_0's l2: 143952
[743]	valid_0's rmse: 379.4	valid_0's l2: 143944
[744]	valid_0's rmse: 379.353	valid

...

[1303]	valid_0's rmse: 372.143	valid_0's l2: 138490
[1304]	valid_0's rmse: 372.144	valid_0's l2: 138491
[1305]	valid_0's rmse: 372.133	valid_0's l2: 138483
[1306]	valid_0's rmse: 372.146	valid_0's l2: 138493
[1307]	valid_0's rmse: 372.145	valid_0's l2: 138492
[1308]	valid_0's rmse: 372.135	valid_0's l2: 138485
[1309]	valid_0's rmse: 372.13	valid_0's l2: 138480
[1310]	valid_0's rmse: 372.119	valid_0's l2: 138473
[1311]	valid_0's rmse: 372.115	valid_0's l2: 138469
[1312]	valid_0's rmse: 372.115	valid_0's l2: 138470
[1313]	valid_0's rmse: 372.09	valid_0's l2: 138451
[1314]	valid_0's rmse: 372.088	valid_0's l2: 138450
[1315]	valid_0's rmse: 372.073	valid_0's l2: 138438
[1316]	valid_0's rmse: 372.064	valid_0's l2: 138431
[1317]	valid_0's rmse: 372.057	valid_0's l2: 138426
[1318]	valid_0's rmse: 372.055	valid_0's l2: 138425
[1319]	valid_0's rmse: 372.05	valid_0's l2: 138421
[1320]	valid_0's rmse: 372.045	valid_0's l2: 138417
[1321]	valid_0's rmse: 372.033	valid_0's l2: 138408
[1322]	valid_0'

[1520]	valid_0's rmse: 370.981	valid_0's l2: 137627
[1521]	valid_0's rmse: 370.976	valid_0's l2: 137623
[1522]	valid_0's rmse: 370.957	valid_0's l2: 137609
[1523]	valid_0's rmse: 370.961	valid_0's l2: 137612
[1524]	valid_0's rmse: 370.951	valid_0's l2: 137604
[1525]	valid_0's rmse: 370.948	valid_0's l2: 137602
[1526]	valid_0's rmse: 370.944	valid_0's l2: 137599
[1527]	valid_0's rmse: 370.942	valid_0's l2: 137598
[1528]	valid_0's rmse: 370.932	valid_0's l2: 137590
[1529]	valid_0's rmse: 370.931	valid_0's l2: 137590
[1530]	valid_0's rmse: 370.925	valid_0's l2: 137585
[1531]	valid_0's rmse: 370.914	valid_0's l2: 137577
[1532]	valid_0's rmse: 370.909	valid_0's l2: 137574
[1533]	valid_0's rmse: 370.897	valid_0's l2: 137565
[1534]	valid_0's rmse: 370.893	valid_0's l2: 137562
[1535]	valid_0's rmse: 370.887	valid_0's l2: 137557
[1536]	valid_0's rmse: 370.888	valid_0's l2: 137558
[1537]	valid_0's rmse: 370.892	valid_0's l2: 137561
[1538]	valid_0's rmse: 370.882	valid_0's l2: 137553
[1539]	valid

[1740]	valid_0's rmse: 370.218	valid_0's l2: 137061
[1741]	valid_0's rmse: 370.22	valid_0's l2: 137063
[1742]	valid_0's rmse: 370.239	valid_0's l2: 137077
[1743]	valid_0's rmse: 370.229	valid_0's l2: 137069
[1744]	valid_0's rmse: 370.235	valid_0's l2: 137074
[1745]	valid_0's rmse: 370.229	valid_0's l2: 137069
[1746]	valid_0's rmse: 370.225	valid_0's l2: 137066
[1747]	valid_0's rmse: 370.227	valid_0's l2: 137068
[1748]	valid_0's rmse: 370.225	valid_0's l2: 137066
[1749]	valid_0's rmse: 370.219	valid_0's l2: 137062
[1750]	valid_0's rmse: 370.218	valid_0's l2: 137062
[1751]	valid_0's rmse: 370.211	valid_0's l2: 137056
[1752]	valid_0's rmse: 370.218	valid_0's l2: 137062
[1753]	valid_0's rmse: 370.212	valid_0's l2: 137057
[1754]	valid_0's rmse: 370.2	valid_0's l2: 137048
[1755]	valid_0's rmse: 370.203	valid_0's l2: 137050
[1756]	valid_0's rmse: 370.211	valid_0's l2: 137056
[1757]	valid_0's rmse: 370.211	valid_0's l2: 137056
[1758]	valid_0's rmse: 370.206	valid_0's l2: 137053
[1759]	valid_0'

[1899]	valid_0's rmse: 369.891	valid_0's l2: 136819
[1900]	valid_0's rmse: 369.885	valid_0's l2: 136815
[1901]	valid_0's rmse: 369.877	valid_0's l2: 136809
[1902]	valid_0's rmse: 369.878	valid_0's l2: 136810
[1903]	valid_0's rmse: 369.868	valid_0's l2: 136802
[1904]	valid_0's rmse: 369.867	valid_0's l2: 136801
[1905]	valid_0's rmse: 369.86	valid_0's l2: 136797
[1906]	valid_0's rmse: 369.859	valid_0's l2: 136795
[1907]	valid_0's rmse: 369.856	valid_0's l2: 136794
[1908]	valid_0's rmse: 369.853	valid_0's l2: 136791
[1909]	valid_0's rmse: 369.853	valid_0's l2: 136792
[1910]	valid_0's rmse: 369.854	valid_0's l2: 136792
[1911]	valid_0's rmse: 369.838	valid_0's l2: 136780
[1912]	valid_0's rmse: 369.829	valid_0's l2: 136774
[1913]	valid_0's rmse: 369.829	valid_0's l2: 136774
[1914]	valid_0's rmse: 369.835	valid_0's l2: 136778
[1915]	valid_0's rmse: 369.827	valid_0's l2: 136772
[1916]	valid_0's rmse: 369.832	valid_0's l2: 136776
[1917]	valid_0's rmse: 369.83	valid_0's l2: 136774
[1918]	valid_0

[2112]	valid_0's rmse: 369.478	valid_0's l2: 136514
[2113]	valid_0's rmse: 369.474	valid_0's l2: 136511
[2114]	valid_0's rmse: 369.474	valid_0's l2: 136511
[2115]	valid_0's rmse: 369.473	valid_0's l2: 136510
[2116]	valid_0's rmse: 369.467	valid_0's l2: 136506
[2117]	valid_0's rmse: 369.467	valid_0's l2: 136506
[2118]	valid_0's rmse: 369.47	valid_0's l2: 136508
[2119]	valid_0's rmse: 369.468	valid_0's l2: 136506
[2120]	valid_0's rmse: 369.471	valid_0's l2: 136509
[2121]	valid_0's rmse: 369.472	valid_0's l2: 136510
[2122]	valid_0's rmse: 369.477	valid_0's l2: 136513
[2123]	valid_0's rmse: 369.482	valid_0's l2: 136517
[2124]	valid_0's rmse: 369.488	valid_0's l2: 136522
[2125]	valid_0's rmse: 369.481	valid_0's l2: 136516
[2126]	valid_0's rmse: 369.506	valid_0's l2: 136535
[2127]	valid_0's rmse: 369.503	valid_0's l2: 136533
[2128]	valid_0's rmse: 369.501	valid_0's l2: 136531
[2129]	valid_0's rmse: 369.497	valid_0's l2: 136528
[2130]	valid_0's rmse: 369.51	valid_0's l2: 136538
[2131]	valid_0

In [35]:
y_pred=lgbm_tuned.predict(x_test)

print("Train setinde rmse hatası:",np.sqrt(mean_squared_error(y_train,lgbm_tuned.predict(x_train))))
print("Test setinde rmse hatası:",np.sqrt(mean_squared_error(y_test,y_pred)))
print("-"*30)
print("Train setinde r2_score:",r2_score(y_train,lgbm_tuned.predict(x_train)))
print("Test setinde r2_score",r2_score(y_test,y_pred))



test_predict=y_pred
train_predict=lgbm_tuned.predict(x_train)

train_report=pd.concat([pd.DataFrame(data=[y_train]).T,pd.DataFrame(data=[pd.Series(train_predict,index=y_train.index,name="price")]).T],axis=1)
test_report=pd.concat([pd.DataFrame(data=[y_test]).T,pd.DataFrame(data=[pd.Series(test_predict,index=y_test.index,name="price")]).T],axis=1)
train_report.columns=["true","pred"]
test_report.columns=["true","pred"]
train_report["error"]=abs(train_report.true-train_report.pred)
test_report["error"]=abs(test_report.true-test_report.pred)

Train setinde rmse hatası: 292.91764324455323
Test setinde rmse hatası: 369.466700548183
------------------------------
Train setinde r2_score: 0.9887677468755937
Test setinde r2_score 0.9822957320479291


In [36]:
## normalize ile train gelişti fakat test geriledi

In [38]:
from sklearn import preprocessing as pp
sed=pp.scale(xtr_backup)
sed_xtr=pd.DataFrame(sed,columns=x_train.columns,index=x_train.index)
sed=pp.normalize(xte_backuo)
sed_xte=pd.DataFrame(sed,columns=x_test.columns,index=x_test.index)

In [40]:
xtr_backup=x_train
xte_backup=x_test

x_train=sed_xtr
x_test=sed_xte

lgbm_tuned=LGBMRegressor(learning_rate=0.01,
                        min_child_samples=20,
                        n_estimators=5000,
                        num_leaves=40).fit(x_train,y_train,eval_set=[(x_test,y_test)],eval_metric= "rmse")


[1]	valid_0's rmse: 2776.97	valid_0's l2: 7.71154e+06
[2]	valid_0's rmse: 2777.06	valid_0's l2: 7.71204e+06
[3]	valid_0's rmse: 2776.55	valid_0's l2: 7.70924e+06
[4]	valid_0's rmse: 2776.67	valid_0's l2: 7.7099e+06
[5]	valid_0's rmse: 2776.76	valid_0's l2: 7.71038e+06
[6]	valid_0's rmse: 2776.3	valid_0's l2: 7.70786e+06
[7]	valid_0's rmse: 2776.4	valid_0's l2: 7.7084e+06
[8]	valid_0's rmse: 2776.46	valid_0's l2: 7.70874e+06
[9]	valid_0's rmse: 2776.55	valid_0's l2: 7.70924e+06
[10]	valid_0's rmse: 2776.72	valid_0's l2: 7.71019e+06
[11]	valid_0's rmse: 2776.93	valid_0's l2: 7.71134e+06
[12]	valid_0's rmse: 2777.07	valid_0's l2: 7.71209e+06
[13]	valid_0's rmse: 2777.29	valid_0's l2: 7.71336e+06
[14]	valid_0's rmse: 2777.44	valid_0's l2: 7.71417e+06
[15]	valid_0's rmse: 2777.67	valid_0's l2: 7.71545e+06
[16]	valid_0's rmse: 2777.93	valid_0's l2: 7.7169e+06
[17]	valid_0's rmse: 2778.06	valid_0's l2: 7.71762e+06
[18]	valid_0's rmse: 2778.16	valid_0's l2: 7.71818e+06
[19]	valid_0's rmse: 277

[170]	valid_0's rmse: 2815.51	valid_0's l2: 7.92707e+06
[171]	valid_0's rmse: 2815.53	valid_0's l2: 7.92719e+06
[172]	valid_0's rmse: 2815.87	valid_0's l2: 7.92914e+06
[173]	valid_0's rmse: 2816.29	valid_0's l2: 7.93148e+06
[174]	valid_0's rmse: 2816.74	valid_0's l2: 7.93401e+06
[175]	valid_0's rmse: 2817.24	valid_0's l2: 7.93685e+06
[176]	valid_0's rmse: 2817.24	valid_0's l2: 7.93683e+06
[177]	valid_0's rmse: 2817.58	valid_0's l2: 7.93875e+06
[178]	valid_0's rmse: 2818.07	valid_0's l2: 7.94154e+06
[179]	valid_0's rmse: 2818.42	valid_0's l2: 7.94348e+06
[180]	valid_0's rmse: 2818.89	valid_0's l2: 7.94617e+06
[181]	valid_0's rmse: 2819.38	valid_0's l2: 7.94891e+06
[182]	valid_0's rmse: 2819.84	valid_0's l2: 7.95148e+06
[183]	valid_0's rmse: 2819.99	valid_0's l2: 7.95232e+06
[184]	valid_0's rmse: 2820.47	valid_0's l2: 7.95506e+06
[185]	valid_0's rmse: 2820.85	valid_0's l2: 7.95717e+06
[186]	valid_0's rmse: 2820.85	valid_0's l2: 7.95719e+06
[187]	valid_0's rmse: 2821.18	valid_0's l2: 7.95

[346]	valid_0's rmse: 2856.42	valid_0's l2: 8.15913e+06
[347]	valid_0's rmse: 2856.45	valid_0's l2: 8.1593e+06
[348]	valid_0's rmse: 2856.61	valid_0's l2: 8.1602e+06
[349]	valid_0's rmse: 2856.68	valid_0's l2: 8.16061e+06
[350]	valid_0's rmse: 2856.74	valid_0's l2: 8.16096e+06
[351]	valid_0's rmse: 2856.86	valid_0's l2: 8.16165e+06
[352]	valid_0's rmse: 2857.01	valid_0's l2: 8.16249e+06
[353]	valid_0's rmse: 2856.92	valid_0's l2: 8.16199e+06
[354]	valid_0's rmse: 2857.06	valid_0's l2: 8.16277e+06
[355]	valid_0's rmse: 2856.97	valid_0's l2: 8.16228e+06
[356]	valid_0's rmse: 2857.13	valid_0's l2: 8.1632e+06
[357]	valid_0's rmse: 2857.23	valid_0's l2: 8.16376e+06
[358]	valid_0's rmse: 2857.35	valid_0's l2: 8.16442e+06
[359]	valid_0's rmse: 2857.53	valid_0's l2: 8.16549e+06
[360]	valid_0's rmse: 2857.54	valid_0's l2: 8.16556e+06
[361]	valid_0's rmse: 2857.6	valid_0's l2: 8.16589e+06
[362]	valid_0's rmse: 2857.67	valid_0's l2: 8.1663e+06
[363]	valid_0's rmse: 2856.67	valid_0's l2: 8.16057e+

[503]	valid_0's rmse: 2866.4	valid_0's l2: 8.21624e+06
[504]	valid_0's rmse: 2866.47	valid_0's l2: 8.21666e+06
[505]	valid_0's rmse: 2866.54	valid_0's l2: 8.21703e+06
[506]	valid_0's rmse: 2866.57	valid_0's l2: 8.21722e+06
[507]	valid_0's rmse: 2866.46	valid_0's l2: 8.21662e+06
[508]	valid_0's rmse: 2866.54	valid_0's l2: 8.21706e+06
[509]	valid_0's rmse: 2866.62	valid_0's l2: 8.2175e+06
[510]	valid_0's rmse: 2866.67	valid_0's l2: 8.2178e+06
[511]	valid_0's rmse: 2866.63	valid_0's l2: 8.21759e+06
[512]	valid_0's rmse: 2866.72	valid_0's l2: 8.21809e+06
[513]	valid_0's rmse: 2866.68	valid_0's l2: 8.21788e+06
[514]	valid_0's rmse: 2866.73	valid_0's l2: 8.21811e+06
[515]	valid_0's rmse: 2866.71	valid_0's l2: 8.21804e+06
[516]	valid_0's rmse: 2866.8	valid_0's l2: 8.21852e+06
[517]	valid_0's rmse: 2866.81	valid_0's l2: 8.2186e+06
[518]	valid_0's rmse: 2866.78	valid_0's l2: 8.21841e+06
[519]	valid_0's rmse: 2866.79	valid_0's l2: 8.21849e+06
[520]	valid_0's rmse: 2866.79	valid_0's l2: 8.21849e+

[679]	valid_0's rmse: 2874.93	valid_0's l2: 8.26525e+06
[680]	valid_0's rmse: 2874.96	valid_0's l2: 8.2654e+06
[681]	valid_0's rmse: 2874.99	valid_0's l2: 8.26557e+06
[682]	valid_0's rmse: 2875.06	valid_0's l2: 8.26597e+06
[683]	valid_0's rmse: 2875.08	valid_0's l2: 8.26607e+06
[684]	valid_0's rmse: 2875.09	valid_0's l2: 8.26616e+06
[685]	valid_0's rmse: 2875.13	valid_0's l2: 8.26639e+06
[686]	valid_0's rmse: 2875.17	valid_0's l2: 8.26661e+06
[687]	valid_0's rmse: 2875.25	valid_0's l2: 8.26706e+06
[688]	valid_0's rmse: 2875.27	valid_0's l2: 8.2672e+06
[689]	valid_0's rmse: 2875.37	valid_0's l2: 8.26777e+06
[690]	valid_0's rmse: 2875.41	valid_0's l2: 8.26799e+06
[691]	valid_0's rmse: 2875.49	valid_0's l2: 8.26846e+06
[692]	valid_0's rmse: 2875.51	valid_0's l2: 8.26855e+06
[693]	valid_0's rmse: 2875.55	valid_0's l2: 8.26877e+06
[694]	valid_0's rmse: 2875.57	valid_0's l2: 8.26888e+06
[695]	valid_0's rmse: 2875.61	valid_0's l2: 8.26912e+06
[696]	valid_0's rmse: 2875.66	valid_0's l2: 8.2694

...

[4162]	valid_0's rmse: 2894.55	valid_0's l2: 8.37842e+06
[4163]	valid_0's rmse: 2894.66	valid_0's l2: 8.37905e+06
[4164]	valid_0's rmse: 2894.66	valid_0's l2: 8.37905e+06
[4165]	valid_0's rmse: 2894.66	valid_0's l2: 8.37905e+06
[4166]	valid_0's rmse: 2894.67	valid_0's l2: 8.3791e+06
[4167]	valid_0's rmse: 2894.66	valid_0's l2: 8.37904e+06
[4168]	valid_0's rmse: 2894.66	valid_0's l2: 8.37904e+06
[4169]	valid_0's rmse: 2894.66	valid_0's l2: 8.37904e+06
[4170]	valid_0's rmse: 2894.66	valid_0's l2: 8.37903e+06
[4171]	valid_0's rmse: 2894.66	valid_0's l2: 8.37904e+06
[4172]	valid_0's rmse: 2894.66	valid_0's l2: 8.37904e+06
[4173]	valid_0's rmse: 2894.66	valid_0's l2: 8.37904e+06
[4174]	valid_0's rmse: 2894.66	valid_0's l2: 8.37905e+06
[4175]	valid_0's rmse: 2894.66	valid_0's l2: 8.37905e+06
[4176]	valid_0's rmse: 2894.66	valid_0's l2: 8.37905e+06
[4177]	valid_0's rmse: 2894.66	valid_0's l2: 8.37905e+06
[4178]	valid_0's rmse: 2894.66	valid_0's l2: 8.37905e+06
[4179]	valid_0's rmse: 2894.66	v

[4327]	valid_0's rmse: 2895.5	valid_0's l2: 8.38394e+06
[4328]	valid_0's rmse: 2895.5	valid_0's l2: 8.38394e+06
[4329]	valid_0's rmse: 2895.5	valid_0's l2: 8.38394e+06
[4330]	valid_0's rmse: 2895.5	valid_0's l2: 8.38394e+06
[4331]	valid_0's rmse: 2895.5	valid_0's l2: 8.38393e+06
[4332]	valid_0's rmse: 2895.5	valid_0's l2: 8.38393e+06
[4333]	valid_0's rmse: 2895.51	valid_0's l2: 8.38398e+06
[4334]	valid_0's rmse: 2895.51	valid_0's l2: 8.38398e+06
[4335]	valid_0's rmse: 2895.51	valid_0's l2: 8.38398e+06
[4336]	valid_0's rmse: 2895.51	valid_0's l2: 8.38398e+06
[4337]	valid_0's rmse: 2895.51	valid_0's l2: 8.38397e+06
[4338]	valid_0's rmse: 2895.51	valid_0's l2: 8.38397e+06
[4339]	valid_0's rmse: 2895.51	valid_0's l2: 8.38397e+06
[4340]	valid_0's rmse: 2895.51	valid_0's l2: 8.38397e+06
[4341]	valid_0's rmse: 2895.51	valid_0's l2: 8.38397e+06
[4342]	valid_0's rmse: 2895.51	valid_0's l2: 8.38397e+06
[4343]	valid_0's rmse: 2895.52	valid_0's l2: 8.38405e+06
[4344]	valid_0's rmse: 2895.52	valid_

[4506]	valid_0's rmse: 2895.45	valid_0's l2: 8.38366e+06
[4507]	valid_0's rmse: 2895.45	valid_0's l2: 8.38366e+06
[4508]	valid_0's rmse: 2895.45	valid_0's l2: 8.38366e+06
[4509]	valid_0's rmse: 2895.44	valid_0's l2: 8.38357e+06
[4510]	valid_0's rmse: 2895.44	valid_0's l2: 8.38357e+06
[4511]	valid_0's rmse: 2895.44	valid_0's l2: 8.38357e+06
[4512]	valid_0's rmse: 2895.44	valid_0's l2: 8.38357e+06
[4513]	valid_0's rmse: 2895.44	valid_0's l2: 8.38357e+06
[4514]	valid_0's rmse: 2895.44	valid_0's l2: 8.38356e+06
[4515]	valid_0's rmse: 2895.44	valid_0's l2: 8.38356e+06
[4516]	valid_0's rmse: 2895.44	valid_0's l2: 8.38356e+06
[4517]	valid_0's rmse: 2895.44	valid_0's l2: 8.38356e+06
[4518]	valid_0's rmse: 2895.44	valid_0's l2: 8.38356e+06
[4519]	valid_0's rmse: 2895.44	valid_0's l2: 8.38356e+06
[4520]	valid_0's rmse: 2895.44	valid_0's l2: 8.38356e+06
[4521]	valid_0's rmse: 2895.44	valid_0's l2: 8.38356e+06
[4522]	valid_0's rmse: 2895.44	valid_0's l2: 8.38356e+06
[4523]	valid_0's rmse: 2895.44	

[4677]	valid_0's rmse: 2895.25	valid_0's l2: 8.38248e+06
[4678]	valid_0's rmse: 2895.25	valid_0's l2: 8.38249e+06
[4679]	valid_0's rmse: 2895.25	valid_0's l2: 8.38249e+06
[4680]	valid_0's rmse: 2895.25	valid_0's l2: 8.38245e+06
[4681]	valid_0's rmse: 2895.25	valid_0's l2: 8.38245e+06
[4682]	valid_0's rmse: 2895.23	valid_0's l2: 8.38237e+06
[4683]	valid_0's rmse: 2895.23	valid_0's l2: 8.38237e+06
[4684]	valid_0's rmse: 2895.23	valid_0's l2: 8.38237e+06
[4685]	valid_0's rmse: 2895.23	valid_0's l2: 8.38236e+06
[4686]	valid_0's rmse: 2895.23	valid_0's l2: 8.38236e+06
[4687]	valid_0's rmse: 2895.23	valid_0's l2: 8.38236e+06
[4688]	valid_0's rmse: 2895.23	valid_0's l2: 8.38236e+06
[4689]	valid_0's rmse: 2895.23	valid_0's l2: 8.38236e+06
[4690]	valid_0's rmse: 2895.23	valid_0's l2: 8.38235e+06
[4691]	valid_0's rmse: 2895.23	valid_0's l2: 8.38237e+06
[4692]	valid_0's rmse: 2895.23	valid_0's l2: 8.38236e+06
[4693]	valid_0's rmse: 2895.23	valid_0's l2: 8.38236e+06
[4694]	valid_0's rmse: 2895.23	

[4849]	valid_0's rmse: 2895.25	valid_0's l2: 8.3825e+06
[4850]	valid_0's rmse: 2895.25	valid_0's l2: 8.3825e+06
[4851]	valid_0's rmse: 2895.25	valid_0's l2: 8.3825e+06
[4852]	valid_0's rmse: 2895.25	valid_0's l2: 8.3825e+06
[4853]	valid_0's rmse: 2895.25	valid_0's l2: 8.3825e+06
[4854]	valid_0's rmse: 2895.25	valid_0's l2: 8.3825e+06
[4855]	valid_0's rmse: 2895.25	valid_0's l2: 8.3825e+06
[4856]	valid_0's rmse: 2895.25	valid_0's l2: 8.38249e+06
[4857]	valid_0's rmse: 2895.25	valid_0's l2: 8.38246e+06
[4858]	valid_0's rmse: 2895.25	valid_0's l2: 8.38246e+06
[4859]	valid_0's rmse: 2895.25	valid_0's l2: 8.38246e+06
[4860]	valid_0's rmse: 2895.25	valid_0's l2: 8.38246e+06
[4861]	valid_0's rmse: 2895.25	valid_0's l2: 8.38246e+06
[4862]	valid_0's rmse: 2895.25	valid_0's l2: 8.38246e+06
[4863]	valid_0's rmse: 2895.25	valid_0's l2: 8.38246e+06
[4864]	valid_0's rmse: 2895.25	valid_0's l2: 8.38246e+06
[4865]	valid_0's rmse: 2895.25	valid_0's l2: 8.38246e+06
[4866]	valid_0's rmse: 2895.25	valid_0

In [42]:
### scaler hiç tutmadı

In [43]:
### belki de dummieslemeden önce scale ve normalize denememiz gerek

In [1]:
### update: scale did not work since i wrote "normalize" whereas i should write "scale", 